In [ ]:
package main

import (
    "errors"
    "fmt"
    "regexp"
    "strings"
    "time"
)

// 用戶註冊系統 - 示範字串錯誤的使用
func main() {
    fmt.Println("=== 字串錯誤示範：用戶註冊系統 ===")
    
    // 測試各種註冊情況
    testCases := []struct {
        username string
        email    string
        age      int
    }{
        {"john_doe", "john@example.com", 25},        // 正常情況
        {"", "john@example.com", 25},                // 缺少用戶名
        {"john", "invalid-email", 25},               // 無效郵箱
        {"admin", "admin@example.com", 16},          // 年齡不符
        {"toolongusernamethatexceedslimit", "user@example.com", 25}, // 用戶名過長
        {"user@123", "user@example.com", 25},        // 用戶名包含非法字符
    }
    
    for i, tc := range testCases {
        fmt.Printf("\n--- 測試案例 %d ---\n", i+1)
        user, err := registerUser(tc.username, tc.email, tc.age)
        if err != nil {
            fmt.Printf("註冊失敗: %v\n", err)
        } else {
            fmt.Printf("註冊成功: %+v\n", user)
        }
    }
    
    fmt.Println("\n=== 展示不同錯誤創建方法 ===")
    demonstrateErrorCreation()
}

// 用戶結構
type User struct {
    ID       int
    Username string
    Email    string
    Age      int
    Created  time.Time
}

// 用戶註冊功能 - 使用多種字串錯誤方式
func registerUser(username, email string, age int) (*User, error) {
    // 1. 使用 errors.New() 建立簡單錯誤
    if username == "" {
        return nil, errors.New("用戶名不能為空")
    }
    
    // 2. 使用 fmt.Errorf() 建立格式化錯誤
    if len(username) > 20 {
        return nil, fmt.Errorf("用戶名太長: %d 字符，最多允許 20 字符", len(username))
    }
    
    if age < 18 {
        return nil, fmt.Errorf("年齡不符合要求: %d 歲，必須至少 18 歲", age)
    }
    
    // 3. 複雜驗證使用 fmt.Errorf() 提供詳細資訊
    if err := validateUsername(username); err != nil {
        return nil, fmt.Errorf("用戶名驗證失敗: %v", err)
    }
    
    if err := validateEmail(email); err != nil {
        return nil, fmt.Errorf("郵箱驗證失敗: %v", err)
    }
    
    // 4. 模擬資料庫檢查，使用格式化錯誤提供上下文
    if err := checkUserExists(username, email); err != nil {
        return nil, err // 已經是格式化的錯誤
    }
    
    // 創建用戶
    user := &User{
        ID:       generateUserID(),
        Username: username,
        Email:    email,
        Age:      age,
        Created:  time.Now(),
    }
    
    return user, nil
}

// 用戶名驗證
func validateUsername(username string) error {
    // 使用正則表達式驗證用戶名格式
    validPattern := regexp.MustCompile(`^[a-zA-Z0-9_]+$`)
    if !validPattern.MatchString(username) {
        return errors.New("只能包含字母、數字和底線")
    }
    
    // 檢查保留字
    reserved := []string{"admin", "root", "system", "guest"}
    for _, word := range reserved {
        if strings.ToLower(username) == word {
            return fmt.Errorf("'%s' 是保留用戶名，不能使用", username)
        }
    }
    
    return nil
}

// 郵箱驗證
func validateEmail(email string) error {
    if email == "" {
        return errors.New("郵箱地址不能為空")
    }
    
    // 簡單的郵箱格式檢查
    emailPattern := regexp.MustCompile(`^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$`)
    if !emailPattern.MatchString(email) {
        return fmt.Errorf("郵箱格式無效: '%s'", email)
    }
    
    return nil
}

// 模擬檢查用戶是否已存在
func checkUserExists(username, email string) error {
    // 模擬已存在的用戶數據
    existingUsers := map[string]string{
        "john_doe":     "john@example.com",
        "existing_user": "existing@example.com",
    }
    
    for existingUsername, existingEmail := range existingUsers {
        if username == existingUsername {
            return fmt.Errorf("用戶名 '%s' 已被使用", username)
        }
        if email == existingEmail {
            return fmt.Errorf("郵箱地址 '%s' 已被註冊", email)
        }
    }
    
    return nil
}

// 生成用戶ID（簡化版）
func generateUserID() int {
    return int(time.Now().UnixNano() % 100000)
}

// 展示不同的錯誤創建方法
func demonstrateErrorCreation() {
    // 1. errors.New() - 靜態錯誤訊息
    staticErr := errors.New("這是一個靜態錯誤訊息")
    fmt.Printf("靜態錯誤: %v\n", staticErr)
    
    // 2. fmt.Errorf() - 格式化錯誤訊息
    username := "test_user"
    maxLength := 20
    formatErr := fmt.Errorf("用戶名 '%s' 長度 %d 超過限制 %d", 
        username, len(username), maxLength)
    fmt.Printf("格式化錯誤: %v\n", formatErr)
    
    // 3. fmt.Errorf() 與多個參數
    multiErr := fmt.Errorf("操作失敗: 嘗試在 %s 創建用戶 %s，錯誤代碼: %d", 
        time.Now().Format("2006-01-02 15:04:05"), "john", 1001)
    fmt.Printf("多參數錯誤: %v\n", multiErr)
    
    // 4. 組合錯誤訊息
    baseErr := errors.New("資料庫連接失敗")
    compositeErr := fmt.Errorf("用戶註冊過程中發生錯誤: %v", baseErr)
    fmt.Printf("組合錯誤: %v\n", compositeErr)
    
    // 5. 條件錯誤訊息
    age := 16
    var conditionalErr error
    if age < 18 {
        conditionalErr = fmt.Errorf("年齡 %d 不符合註冊要求（需要 >= 18）", age)
    } else if age > 120 {
        conditionalErr = fmt.Errorf("年齡 %d 似乎不太合理（需要 <= 120）", age)
    }
    
    if conditionalErr != nil {
        fmt.Printf("條件錯誤: %v\n", conditionalErr)
    }
}

## 2. 用字串來表示簡單的錯誤

Go 提供了幾種建立簡單字串錯誤的方法。這些方法適用於不需要額外資訊的基本錯誤情況。

### errors.New()
最簡單的方法是使用 `errors.New()` 建立一個包含錯誤訊息的錯誤值。

### fmt.Errorf()
`fmt.Errorf()` 提供格式化功能，讓你可以建立包含動態資訊的錯誤訊息。從 Go 1.13 開始，它還支援錯誤包裝功能。

#### 使用場景：
- **簡單驗證錯誤**: 輸入格式不正確
- **業務邏輯錯誤**: 違反業務規則
- **資源不存在**: 找不到指定的資源
- **格式化錯誤訊息**: 包含具體的錯誤詳情

#### 最佳實踐：
- 錯誤訊息應該清楚描述問題
- 使用小寫開頭，不要以句號結尾
- 包含足夠的上下文資訊幫助除錯
- 對於用戶可見的錯誤，考慮國際化需求

In [ ]:
package main

import (
    "fmt"
    "strconv"
    "time"
    "math/rand"
    "net/http"
    "io"
)

// 示範基本錯誤處理模式
func main() {
    fmt.Println("=== 基本錯誤處理示範 ===")
    
    // 1. 檢查並返回模式（最常見）
    fmt.Println("\n1. 檢查並返回模式:")
    result, err := parseAndDouble("42")
    if err != nil {
        fmt.Printf("處理失敗: %v\n", err)
        return // 提早返回
    }
    fmt.Printf("成功: %d\n", result)
    
    // 測試錯誤情況
    _, err = parseAndDouble("abc")
    if err != nil {
        fmt.Printf("預期的錯誤: %v\n", err)
    }
    
    // 2. 檢查並記錄模式
    fmt.Println("\n2. 檢查並記錄模式:")
    processMultipleValues([]string{"10", "20", "xyz", "30"})
    
    // 3. 檢查並重試模式
    fmt.Println("\n3. 檢查並重試模式:")
    data, err := fetchWithRetry("https://httpbin.org/status/200", 3)
    if err != nil {
        fmt.Printf("最終失敗: %v\n", err)
    } else {
        fmt.Printf("成功獲取 %d 字節\n", len(data))
    }
    
    // 4. 檢查並降級模式
    fmt.Println("\n4. 檢查並降級模式:")
    greeting := getGreetingWithFallback("John")
    fmt.Println(greeting)
}

// 檢查並返回：最基本的錯誤處理模式
func parseAndDouble(s string) (int, error) {
    // strconv.Atoi 返回 (int, error)
    num, err := strconv.Atoi(s)
    if err != nil {
        // 直接返回錯誤，讓調用者處理
        return 0, err
    }
    return num * 2, nil
}

// 檢查並記錄：記錄錯誤但繼續處理其他項目
func processMultipleValues(values []string) {
    successCount := 0
    for i, value := range values {
        result, err := parseAndDouble(value)
        if err != nil {
            // 記錄錯誤但繼續處理下一個
            fmt.Printf("  項目 %d (%s) 處理失敗: %v\n", i, value, err)
            continue
        }
        fmt.Printf("  項目 %d: %s -> %d\n", i, value, result)
        successCount++
    }
    fmt.Printf("  總共處理成功 %d 個項目\n", successCount)
}

// 檢查並重試：對於可能暫時失敗的操作
func fetchWithRetry(url string, maxRetries int) ([]byte, error) {
    var lastErr error
    
    for attempt := 1; attempt <= maxRetries; attempt++ {
        fmt.Printf("  嘗試 %d/%d: 獲取 %s\n", attempt, maxRetries, url)
        
        resp, err := http.Get(url)
        if err != nil {
            lastErr = err
            fmt.Printf("  請求失敗: %v\n", err)
            if attempt < maxRetries {
                time.Sleep(time.Duration(attempt) * time.Second) // 指數退避
            }
            continue
        }
        defer resp.Body.Close()
        
        if resp.StatusCode != 200 {
            lastErr = fmt.Errorf("HTTP錯誤: %d", resp.StatusCode)
            fmt.Printf("  HTTP錯誤: %d\n", resp.StatusCode)
            if attempt < maxRetries {
                time.Sleep(time.Duration(attempt) * time.Second)
            }
            continue
        }
        
        // 成功！讀取回應內容
        data, err := io.ReadAll(resp.Body)
        if err != nil {
            lastErr = err
            continue
        }
        
        return data, nil
    }
    
    return nil, fmt.Errorf("所有重試都失敗了，最後錯誤: %w", lastErr)
}

// 檢查並降級：提供替代方案
func getGreetingWithFallback(name string) string {
    // 模擬可能失敗的個性化問候服務
    personalizedGreeting, err := getPersonalizedGreeting(name)
    if err != nil {
        // 降級到通用問候
        fmt.Printf("  個性化問候失敗: %v, 使用預設問候\n", err)
        return fmt.Sprintf("Hello, %s!", name)
    }
    return personalizedGreeting
}

// 模擬可能失敗的服務
func getPersonalizedGreeting(name string) (string, error) {
    // 模擬隨機失敗
    rand.Seed(time.Now().UnixNano())
    if rand.Float32() < 0.3 { // 30% 失敗率
        return "", fmt.Errorf("個性化服務暫時不可用")
    }
    
    greetings := map[string]string{
        "John":  "Hey there, John! How's your day going?",
        "Alice": "Good morning, Alice! Ready for another great day?",
        "Bob":   "Hi Bob! Nice to see you again!",
    }
    
    if greeting, exists := greetings[name]; exists {
        return greeting, nil
    }
    
    return fmt.Sprintf("Welcome, %s! Great to meet you!", name), nil
}

## 1. 如何處理錯誤：基本知識

在 Go 中，錯誤是一個內建的介面型態，定義非常簡單：

```go
type error interface {
    Error() string
}
```

### 基本錯誤處理模式

Go 的函式通常會回傳兩個值：一個是期望的結果，另一個是 error。這種模式讓錯誤處理變得明確且一致。

#### 核心概念：
- **錯誤是值**: 可以被儲存、傳遞、檢查
- **零值代表成功**: error 的零值是 nil，代表沒有錯誤
- **必須檢查**: Go 編譯器會警告未使用的錯誤值
- **提早返回**: 遇到錯誤時通常立即處理並返回

#### 常見的錯誤處理模式：
1. **檢查並返回**: 最常見的模式
2. **檢查並記錄**: 記錄錯誤但繼續執行
3. **檢查並重試**: 對於可能暫時失敗的操作
4. **檢查並降級**: 提供替代方案

# 第八章 錯誤處理 (Error Handling)

## 章節概述

錯誤處理是 Go 程式設計的核心概念之一。與許多其他程式語言使用例外處理 (exception handling) 不同，Go 採用明確的錯誤值回傳機制。這種設計哲學讓錯誤處理變得明確且可預測，但也要求開發者更加謹慎地處理每一個可能的錯誤狀況。

### 本章重點內容
- **錯誤處理基本概念**: 理解 Go 的錯誤處理哲學
- **簡單字串錯誤**: 使用 errors.New 和 fmt.Errorf 建立錯誤
- **哨兵錯誤 (Sentinel Errors)**: 預定義的錯誤值用於特定狀況
- **自訂錯誤型態**: 建立包含額外資訊的錯誤型態
- **錯誤包裝 (Error Wrapping)**: 在錯誤傳遞過程中保留上下文
- **errors.Is 與 errors.As**: 檢查和轉換包裝的錯誤
- **defer 與錯誤處理**: 使用 defer 進行清理和錯誤處理
- **panic 與 recover**: 緊急狀況的處理機制
- **堆疊追蹤**: 取得錯誤的詳細資訊

### Go 錯誤處理的設計原則
1. **明確性**: 錯誤必須被明確處理，不能忽略
2. **簡潔性**: 錯誤是值，可以像其他值一樣操作
3. **可組合性**: 錯誤可以被包裝和組合以提供更多上下文

# 第八章 錯誤 (Error Handling)

錯誤處理是 Go 程式設計的核心概念之一。Go 採用了獨特的錯誤處理方式，不使用傳統的例外機制，而是將錯誤視為值來處理。本章將深入探討 Go 的錯誤處理哲學和最佳實踐。

## 本章重點內容
- 如何處理錯誤：基本知識
- 用字串來表示簡單的錯誤
- 哨兵錯誤
- error 是值
- 包裝錯誤
- Is 與 As
- 用 defer 來包裝 error
- panic 與 recover
- 從 error 取得堆疊追蹤

## 1. 如何處理錯誤：基礎

在 Go 中，錯誤是一個內建的介面型態。函式通常回傳兩個值：結果和錯誤。如果沒有錯誤，錯誤值為 `nil`。

### error 介面

```go
type error interface {
    Error() string
}
```

### 基本錯誤處理模式

Go 的錯誤處理哲學是「錯誤是值」，與其他語言的例外機制不同：

**錯誤檢查模式**: Go 使用明確的錯誤檢查，而不是 try-catch 機制。

**早期返回**: 遇到錯誤立即檢查並處理，通常會提早返回。

**錯誤傳播**: 錯誤通常會向上層傳播，每一層都可以添加上下文資訊。

**nil 代表成功**: 當沒有錯誤時，error 值為 nil。

**慣用模式**: `if err != nil` 是 Go 中最常見的錯誤檢查模式。

基本範例：
```go
result, err := someFunction()
if err != nil {
    return fmt.Errorf("operation failed: %w", err)
}
// 繼續處理 result
```

In [ ]:
package main

import (
    "fmt"
    "strconv"
    "os"
)

// 簡單的除法函式，展示基本錯誤處理
func divide(a, b float64) (float64, error) {
    if b == 0 {
        return 0, fmt.Errorf("division by zero: cannot divide %f by %f", a, b)
    }
    return a / b, nil  // 成功時回傳 nil 錯誤
}

// 解析數字的函式
func parseAndDivide(aStr, bStr string) (float64, error) {
    // 解析第一個數字
    a, err := strconv.ParseFloat(aStr, 64)
    if err != nil {
        return 0, fmt.Errorf("failed to parse first number '%s': %w", aStr, err)
    }
    
    // 解析第二個數字
    b, err := strconv.ParseFloat(bStr, 64)
    if err != nil {
        return 0, fmt.Errorf("failed to parse second number '%s': %w", bStr, err)
    }
    
    // 執行除法
    result, err := divide(a, b)
    if err != nil {
        return 0, fmt.Errorf("division failed: %w", err)
    }
    
    return result, nil
}

// 讀取檔案的函式
func readFileSize(filename string) (int64, error) {
    info, err := os.Stat(filename)
    if err != nil {
        return 0, fmt.Errorf("failed to get file info for '%s': %w", filename, err)
    }
    return info.Size(), nil
}

func main() {
    fmt.Println("=== 基本錯誤處理範例 ===")
    
    // 1. 成功的除法
    result, err := divide(10, 2)
    if err != nil {
        fmt.Printf("Error: %v\n", err)
    } else {
        fmt.Printf("10 / 2 = %.2f\n", result)
    }
    
    // 2. 除零錯誤
    result, err = divide(10, 0)
    if err != nil {
        fmt.Printf("Error: %v\n", err)
    } else {
        fmt.Printf("Result: %.2f\n", result)
    }
    
    fmt.Println("\n=== 字串解析錯誤處理 ===")
    
    // 3. 成功的字串解析和除法
    testCases := [][]string{
        {"20", "4"},      // 成功
        {"abc", "5"},     // 第一個數字解析失敗
        {"15", "xyz"},    // 第二個數字解析失敗
        {"10", "0"},      // 除零錯誤
    }
    
    for i, testCase := range testCases {
        fmt.Printf("\nTest %d: %s / %s\n", i+1, testCase[0], testCase[1])
        result, err := parseAndDivide(testCase[0], testCase[1])
        if err != nil {
            fmt.Printf("  Error: %v\n", err)
        } else {
            fmt.Printf("  Result: %.2f\n", result)
        }
    }
    
    fmt.Println("\n=== 檔案操作錯誤處理 ===")
    
    // 4. 檔案操作錯誤
    files := []string{
        "go.mod",           // 可能存在的檔案
        "nonexistent.txt",  // 不存在的檔案
    }
    
    for _, filename := range files {
        fmt.Printf("\nChecking file: %s\n", filename)
        size, err := readFileSize(filename)
        if err != nil {
            fmt.Printf("  Error: %v\n", err)
        } else {
            fmt.Printf("  File size: %d bytes\n", size)
        }
    }
    
    fmt.Println("\n=== 錯誤處理的基本原則 ===")
    fmt.Println("1. 總是檢查錯誤：if err != nil")
    fmt.Println("2. 錯誤優先回傳：return result, error")
    fmt.Println("3. 提供上下文：包裝錯誤並添加有用資訊")
    fmt.Println("4. 早期回傳：遇到錯誤立即處理")
}

## 2. 使用字串來建立錯誤

Go 提供了多種建立錯誤的方式。最簡單的是使用 `errors.New()` 和 `fmt.Errorf()`。

### 建立錯誤的方法

In [ ]:
package main

import (
    "errors"
    "fmt"
    "strings"
    "unicode"
)

// 使用 errors.New 建立簡單錯誤
var (
    ErrEmptyString = errors.New("string cannot be empty")
    ErrInvalidInput = errors.New("invalid input provided")
    ErrNotFound = errors.New("item not found")
)

// 驗證使用者名稱
func validateUsername(username string) error {
    if username == "" {
        return ErrEmptyString  // 回傳預定義的錯誤
    }
    
    if len(username) < 3 {
        return fmt.Errorf("username too short: got %d characters, need at least 3", len(username))
    }
    
    if len(username) > 20 {
        return fmt.Errorf("username too long: got %d characters, maximum is 20", len(username))
    }
    
    // 檢查是否包含非法字元
    for i, r := range username {
        if !unicode.IsLetter(r) && !unicode.IsDigit(r) && r != '_' {
            return fmt.Errorf("invalid character '%c' at position %d: only letters, digits, and underscore allowed", r, i)
        }
    }
    
    return nil  // 驗證成功
}

// 驗證密碼強度
func validatePassword(password string) error {
    if password == "" {
        return errors.New("password cannot be empty")
    }
    
    if len(password) < 8 {
        return fmt.Errorf("password too weak: need at least 8 characters, got %d", len(password))
    }
    
    var (
        hasUpper   = false
        hasLower   = false
        hasDigit   = false
        hasSpecial = false
    )
    
    for _, r := range password {
        switch {
        case unicode.IsUpper(r):
            hasUpper = true
        case unicode.IsLower(r):
            hasLower = true
        case unicode.IsDigit(r):
            hasDigit = true
        case strings.ContainsRune("!@#$%^&*()_+-=[]{}|;:,.<>?", r):
            hasSpecial = true
        }
    }
    
    var missing []string
    if !hasUpper {
        missing = append(missing, "uppercase letter")
    }
    if !hasLower {
        missing = append(missing, "lowercase letter")
    }
    if !hasDigit {
        missing = append(missing, "digit")
    }
    if !hasSpecial {
        missing = append(missing, "special character")
    }
    
    if len(missing) > 0 {
        return fmt.Errorf("password missing: %s", strings.Join(missing, ", "))
    }
    
    return nil
}

// 組合驗證函式
func validateUser(username, password string) error {
    if err := validateUsername(username); err != nil {
        return fmt.Errorf("username validation failed: %w", err)
    }
    
    if err := validatePassword(password); err != nil {
        return fmt.Errorf("password validation failed: %w", err)
    }
    
    return nil
}

// 模擬資料庫操作
func findUserByID(id int) (string, error) {
    users := map[int]string{
        1: "alice",
        2: "bob",
        3: "charlie",
    }
    
    if id <= 0 {
        return "", fmt.Errorf("invalid user ID: %d (must be positive)", id)
    }
    
    username, exists := users[id]
    if !exists {
        return "", fmt.Errorf("user not found: no user with ID %d", id)
    }
    
    return username, nil
}

func main() {
    fmt.Println("=== 字串錯誤建立範例 ===")
    
    // 1. 使用者名稱驗證測試
    fmt.Println("\n1. 使用者名稱驗證:")
    usernames := []string{
        "alice",        // 有效
        "",             // 空字串
        "ab",           // 太短
        "verylongusernamethatexceedslimit", // 太長
        "user@name",    // 非法字元
        "valid_user123", // 有效
    }
    
    for _, username := range usernames {
        fmt.Printf("  Testing '%s': ", username)
        if err := validateUsername(username); err != nil {
            fmt.Printf("❌ %v\n", err)
        } else {
            fmt.Printf("✅ Valid\n")
        }
    }
    
    // 2. 密碼驗證測試
    fmt.Println("\n2. 密碼驗證:")
    passwords := []string{
        "Password123!",  // 有效
        "",              // 空字串
        "short",         // 太短
        "nouppercase123!", // 缺少大寫
        "NOLOWERCASE123!", // 缺少小寫
        "NoDigits!",      // 缺少數字
        "NoSpecial123",   // 缺少特殊字元
        "onlylowercase",  // 多項缺失
    }
    
    for _, password := range passwords {
        fmt.Printf("  Testing password: ")
        if err := validatePassword(password); err != nil {
            fmt.Printf("❌ %v\n", err)
        } else {
            fmt.Printf("✅ Strong password\n")
        }
    }
    
    // 3. 組合驗證測試
    fmt.Println("\n3. 完整使用者驗證:")
    testUsers := [][]string{
        {"alice", "Password123!"},
        {"", "Password123!"},
        {"bob", "weak"},
        {"invalid@", "short"},
    }
    
    for i, user := range testUsers {
        fmt.Printf("  User %d ('%s'): ", i+1, user[0])
        if err := validateUser(user[0], user[1]); err != nil {
            fmt.Printf("❌ %v\n", err)
        } else {
            fmt.Printf("✅ Valid user\n")
        }
    }
    
    // 4. 資料庫查詢測試
    fmt.Println("\n4. 資料庫查詢:")
    ids := []int{1, 2, 5, -1, 0}
    
    for _, id := range ids {
        fmt.Printf("  Finding user ID %d: ", id)
        username, err := findUserByID(id)
        if err != nil {
            fmt.Printf("❌ %v\n", err)
        } else {
            fmt.Printf("✅ Found user: %s\n", username)
        }
    }
    
    fmt.Println("\n=== 錯誤建立最佳實踐 ===")
    fmt.Println("✓ 使用 errors.New() 建立簡單的常數錯誤")
    fmt.Println("✓ 使用 fmt.Errorf() 建立包含動態資訊的錯誤")
    fmt.Println("✓ 錯誤訊息應該具體且有幫助")
    fmt.Println("✓ 包含足夠的上下文資訊協助除錯")
    fmt.Println("✓ 使用 %w 動詞來包裝錯誤")
}

## 3. Sentinel 錯誤

Sentinel 錯誤是預先定義的錯誤值，用於表示特定的錯誤條件。這允許呼叫者檢查特定的錯誤類型。

### Sentinel 錯誤的定義和使用

In [ ]:
package main

import (
    "errors"
    "fmt"
    "io"
)

// Sentinel 錯誤定義
var (
    ErrNotFound       = errors.New("item not found")
    ErrAlreadyExists  = errors.New("item already exists")
    ErrInvalidInput   = errors.New("invalid input")
    ErrUnauthorized   = errors.New("unauthorized access")
    ErrServiceUnavailable = errors.New("service unavailable")
    ErrTimeout        = errors.New("operation timeout")
)

// 模擬使用者儲存庫
type UserRepository struct {
    users map[string]User
}

type User struct {
    ID    string
    Name  string
    Email string
}

func NewUserRepository() *UserRepository {
    return &UserRepository{
        users: map[string]User{
            "alice": {ID: "alice", Name: "Alice Smith", Email: "alice@example.com"},
            "bob":   {ID: "bob", Name: "Bob Johnson", Email: "bob@example.com"},
        },
    }
}

func (ur *UserRepository) GetUser(id string) (User, error) {
    if id == "" {
        return User{}, ErrInvalidInput
    }
    
    user, exists := ur.users[id]
    if !exists {
        return User{}, ErrNotFound
    }
    
    return user, nil
}

func (ur *UserRepository) CreateUser(user User) error {
    if user.ID == "" || user.Name == "" {
        return ErrInvalidInput
    }
    
    if _, exists := ur.users[user.ID]; exists {
        return ErrAlreadyExists
    }
    
    ur.users[user.ID] = user
    return nil
}

func (ur *UserRepository) UpdateUser(user User) error {
    if user.ID == "" {
        return ErrInvalidInput
    }
    
    if _, exists := ur.users[user.ID]; !exists {
        return ErrNotFound
    }
    
    ur.users[user.ID] = user
    return nil
}

func (ur *UserRepository) DeleteUser(id string) error {
    if id == "" {
        return ErrInvalidInput
    }
    
    if _, exists := ur.users[id]; !exists {
        return ErrNotFound
    }
    
    delete(ur.users, id)
    return nil
}

// 模擬網路服務
type NetworkService struct {
    isOnline bool
}

func (ns *NetworkService) FetchData(url string) ([]byte, error) {
    if url == "" {
        return nil, ErrInvalidInput
    }
    
    if !ns.isOnline {
        return nil, ErrServiceUnavailable
    }
    
    // 模擬不同的錯誤情況
    switch url {
    case "http://timeout.com":
        return nil, ErrTimeout
    case "http://unauthorized.com":
        return nil, ErrUnauthorized
    case "http://notfound.com":
        return nil, ErrNotFound
    default:
        return []byte("mock data from " + url), nil
    }
}

// 處理不同錯誤的函式
func handleUserOperation(repo *UserRepository, operation string, userID string) {
    fmt.Printf("\n--- %s for user '%s' ---\n", operation, userID)
    
    user, err := repo.GetUser(userID)
    
    // 使用 Sentinel 錯誤進行特定處理
    switch {
    case errors.Is(err, ErrNotFound):
        fmt.Printf("❌ User not found: %s\n", userID)
        fmt.Println("💡 Suggestion: Check the user ID or create a new user")
        
    case errors.Is(err, ErrInvalidInput):
        fmt.Printf("❌ Invalid input: %s\n", userID)
        fmt.Println("💡 Suggestion: Provide a valid user ID")
        
    case err != nil:
        fmt.Printf("❌ Unexpected error: %v\n", err)
        
    default:
        fmt.Printf("✅ User found: %+v\n", user)
    }
}

func handleNetworkOperation(service *NetworkService, url string) {
    fmt.Printf("\n--- Fetching data from %s ---\n", url)
    
    data, err := service.FetchData(url)
    
    // 根據不同的 Sentinel 錯誤提供不同的處理
    switch {
    case errors.Is(err, ErrTimeout):
        fmt.Println("❌ Request timeout")
        fmt.Println("🔄 Suggestion: Retry the request")
        
    case errors.Is(err, ErrUnauthorized):
        fmt.Println("❌ Unauthorized access")
        fmt.Println("🔑 Suggestion: Check authentication credentials")
        
    case errors.Is(err, ErrServiceUnavailable):
        fmt.Println("❌ Service unavailable")
        fmt.Println("⏳ Suggestion: Try again later")
        
    case errors.Is(err, ErrNotFound):
        fmt.Println("❌ Resource not found")
        fmt.Println("🔍 Suggestion: Check the URL")
        
    case errors.Is(err, ErrInvalidInput):
        fmt.Println("❌ Invalid URL")
        fmt.Println("📝 Suggestion: Provide a valid URL")
        
    case err != nil:
        fmt.Printf("❌ Unexpected error: %v\n", err)
        
    default:
        fmt.Printf("✅ Data fetched successfully: %s\n", string(data))
    }
}

func main() {
    fmt.Println("=== Sentinel 錯誤範例 ===")
    
    // 1. 使用者儲存庫操作
    fmt.Println("\n1. 使用者儲存庫操作:")
    repo := NewUserRepository()
    
    // 測試不同的使用者 ID
    userIDs := []string{"alice", "charlie", "", "nonexistent"}
    for _, userID := range userIDs {
        handleUserOperation(repo, "Get User", userID)
    }
    
    // 2. 建立使用者操作
    fmt.Println("\n2. 建立使用者操作:")
    testUsers := []User{
        {ID: "charlie", Name: "Charlie Brown", Email: "charlie@example.com"}, // 新使用者
        {ID: "alice", Name: "Alice Updated", Email: "alice.new@example.com"}, // 已存在
        {ID: "", Name: "No ID", Email: "noid@example.com"},                    // 無效輸入
    }
    
    for _, user := range testUsers {
        fmt.Printf("\n--- Creating user '%s' ---\n", user.ID)
        err := repo.CreateUser(user)
        
        switch {
        case errors.Is(err, ErrAlreadyExists):
            fmt.Printf("❌ User already exists: %s\n", user.ID)
            fmt.Println("💡 Suggestion: Use update operation instead")
            
        case errors.Is(err, ErrInvalidInput):
            fmt.Println("❌ Invalid user data")
            fmt.Println("💡 Suggestion: Provide valid ID and name")
            
        case err != nil:
            fmt.Printf("❌ Unexpected error: %v\n", err)
            
        default:
            fmt.Printf("✅ User created successfully: %s\n", user.ID)
        }
    }
    
    // 3. 網路服務操作
    fmt.Println("\n3. 網路服務操作:")
    service := &NetworkService{isOnline: true}
    
    urls := []string{
        "http://example.com",
        "http://timeout.com",
        "http://unauthorized.com",
        "http://notfound.com",
        "",
    }
    
    for _, url := range urls {
        handleNetworkOperation(service, url)
    }
    
    // 4. 服務離線情況
    fmt.Println("\n4. 服務離線情況:")
    service.isOnline = false
    handleNetworkOperation(service, "http://example.com")
    
    // 5. 與標準函式庫的 Sentinel 錯誤比較
    fmt.Println("\n5. 標準函式庫 Sentinel 錯誤:")
    fmt.Printf("io.EOF: %v\n", io.EOF)
    fmt.Printf("Is io.EOF a sentinel error? %t\n", errors.Is(io.EOF, io.EOF))
    
    fmt.Println("\n=== Sentinel 錯誤的優點 ===")
    fmt.Println("✓ 允許呼叫者檢查特定錯誤類型")
    fmt.Println("✓ 提供一致的錯誤處理")
    fmt.Println("✓ 支援不同的錯誤處理策略")
    fmt.Println("✓ 容易測試和模擬")
    fmt.Println("✓ 自我描述的錯誤條件")
}

## 4. 錯誤是值

在 Go 中，錯誤就是普通的值。我們可以建立自訂的錯誤型態來攜帶更多資訊。

### 自訂錯誤型態

In [ ]:
package main

import (
    "fmt"
    "strconv"
    "time"
)

// 自訂錯誤型態 - 驗證錯誤
type ValidationError struct {
    Field   string    // 欄位名稱
    Value   interface{} // 錯誤的值
    Message string    // 錯誤訊息
}

func (ve ValidationError) Error() string {
    return fmt.Sprintf("validation failed for field '%s': %s (value: %v)", 
                      ve.Field, ve.Message, ve.Value)
}

// 自訂錯誤型態 - 網路錯誤
type NetworkError struct {
    Operation string    // 操作類型
    URL       string    // 請求的 URL
    Code      int       // HTTP 狀態碼
    Timestamp time.Time // 錯誤發生時間
    Retryable bool      // 是否可重試
}

func (ne NetworkError) Error() string {
    return fmt.Sprintf("%s failed for %s: HTTP %d (at %s)", 
                      ne.Operation, ne.URL, ne.Code, ne.Timestamp.Format(time.RFC3339))
}

func (ne NetworkError) IsRetryable() bool {
    return ne.Retryable
}

// 自訂錯誤型態 - 業務邏輯錯誤
type BusinessError struct {
    Code    string                 // 錯誤代碼
    Message string                 // 使用者友善的錯誤訊息
    Details map[string]interface{} // 額外的錯誤詳情
}

func (be BusinessError) Error() string {
    if len(be.Details) == 0 {
        return fmt.Sprintf("[%s] %s", be.Code, be.Message)
    }
    return fmt.Sprintf("[%s] %s (details: %+v)", be.Code, be.Message, be.Details)
}

func (be BusinessError) GetCode() string {
    return be.Code
}

func (be BusinessError) GetDetails() map[string]interface{} {
    return be.Details
}

// 多重錯誤型態
type MultiError struct {
    Errors []error
}

func (me MultiError) Error() string {
    if len(me.Errors) == 0 {
        return "no errors"
    }
    if len(me.Errors) == 1 {
        return me.Errors[0].Error()
    }
    
    result := fmt.Sprintf("%d errors occurred:", len(me.Errors))
    for i, err := range me.Errors {
        result += fmt.Sprintf("\n  %d. %s", i+1, err.Error())
    }
    return result
}

func (me *MultiError) Add(err error) {
    if err != nil {
        me.Errors = append(me.Errors, err)
    }
}

func (me MultiError) HasErrors() bool {
    return len(me.Errors) > 0
}

// 使用自訂錯誤的函式
func validateAge(age interface{}) error {
    // 先檢查是否為整數
    var ageInt int
    var ok bool
    
    switch v := age.(type) {
    case int:
        ageInt = v
        ok = true
    case string:
        if parsed, err := strconv.Atoi(v); err == nil {
            ageInt = parsed
            ok = true
        }
    }
    
    if !ok {
        return ValidationError{
            Field:   "age",
            Value:   age,
            Message: "must be a valid integer",
        }
    }
    
    if ageInt < 0 {
        return ValidationError{
            Field:   "age",
            Value:   ageInt,
            Message: "cannot be negative",
        }
    }
    
    if ageInt > 150 {
        return ValidationError{
            Field:   "age",
            Value:   ageInt,
            Message: "seems unrealistic (>150)",
        }
    }
    
    return nil
}

func fetchURL(url string) ([]byte, error) {
    // 模擬網路請求
    switch url {
    case "http://success.com":
        return []byte("success data"), nil
        
    case "http://notfound.com":
        return nil, NetworkError{
            Operation: "GET",
            URL:       url,
            Code:      404,
            Timestamp: time.Now(),
            Retryable: false,
        }
        
    case "http://timeout.com":
        return nil, NetworkError{
            Operation: "GET",
            URL:       url,
            Code:      408,
            Timestamp: time.Now(),
            Retryable: true,
        }
        
    case "http://server-error.com":
        return nil, NetworkError{
            Operation: "GET",
            URL:       url,
            Code:      500,
            Timestamp: time.Now(),
            Retryable: true,
        }
        
    default:
        return nil, fmt.Errorf("unknown URL: %s", url)
    }
}

func processOrder(orderID string, amount float64, customerID int) error {
    if orderID == "" {
        return BusinessError{
            Code:    "INVALID_ORDER_ID",
            Message: "Order ID cannot be empty",
        }
    }
    
    if amount <= 0 {
        return BusinessError{
            Code:    "INVALID_AMOUNT",
            Message: "Order amount must be positive",
            Details: map[string]interface{}{
                "provided_amount": amount,
                "minimum_amount":  0.01,
            },
        }
    }
    
    if customerID <= 0 {
        return BusinessError{
            Code:    "INVALID_CUSTOMER",
            Message: "Customer ID must be positive",
            Details: map[string]interface{}{
                "provided_customer_id": customerID,
            },
        }
    }
    
    // 模擬特殊業務規則
    if amount > 10000 {
        return BusinessError{
            Code:    "AMOUNT_LIMIT_EXCEEDED",
            Message: "Order amount exceeds daily limit",
            Details: map[string]interface{}{
                "amount":     amount,
                "daily_limit": 10000,
                "suggestion": "Split into multiple orders or contact support",
            },
        }
    }
    
    return nil
}

func validateUserData(data map[string]interface{}) error {
    var multiErr MultiError
    
    // 驗證名稱
    if name, exists := data["name"]; !exists {
        multiErr.Add(ValidationError{
            Field:   "name",
            Value:   nil,
            Message: "is required",
        })
    } else if nameStr, ok := name.(string); !ok {
        multiErr.Add(ValidationError{
            Field:   "name",
            Value:   name,
            Message: "must be a string",
        })
    } else if len(nameStr) < 2 {
        multiErr.Add(ValidationError{
            Field:   "name",
            Value:   nameStr,
            Message: "must be at least 2 characters",
        })
    }
    
    // 驗證年齡
    if age, exists := data["age"]; exists {
        if err := validateAge(age); err != nil {
            multiErr.Add(err)
        }
    }
    
    // 驗證電子郵件
    if email, exists := data["email"]; exists {
        if emailStr, ok := email.(string); !ok {
            multiErr.Add(ValidationError{
                Field:   "email",
                Value:   email,
                Message: "must be a string",
            })
        } else if emailStr != "" && !containsAt(emailStr) {
            multiErr.Add(ValidationError{
                Field:   "email",
                Value:   emailStr,
                Message: "must contain @ symbol",
            })
        }
    }
    
    if multiErr.HasErrors() {
        return multiErr
    }
    
    return nil
}

func containsAt(s string) bool {
    for _, r := range s {
        if r == '@' {
            return true
        }
    }
    return false
}

func main() {
    fmt.Println("=== 自訂錯誤型態範例 ===")
    
    // 1. 驗證錯誤測試
    fmt.Println("\n1. 驗證錯誤測試:")
    ages := []interface{}{25, "30", -5, 200, "invalid", 3.14}
    
    for _, age := range ages {
        fmt.Printf("  Testing age %v (%T): ", age, age)
        if err := validateAge(age); err != nil {
            if ve, ok := err.(ValidationError); ok {
                fmt.Printf("❌ ValidationError - Field: %s, Message: %s\n", ve.Field, ve.Message)
            } else {
                fmt.Printf("❌ %v\n", err)
            }
        } else {
            fmt.Println("✅ Valid")
        }
    }
    
    // 2. 網路錯誤測試
    fmt.Println("\n2. 網路錯誤測試:")
    urls := []string{
        "http://success.com",
        "http://notfound.com",
        "http://timeout.com",
        "http://server-error.com",
    }
    
    for _, url := range urls {
        fmt.Printf("  Fetching %s: ", url)
        data, err := fetchURL(url)
        if err != nil {
            if ne, ok := err.(NetworkError); ok {
                fmt.Printf("❌ NetworkError - Code: %d, Retryable: %t\n", ne.Code, ne.IsRetryable())
                if ne.IsRetryable() {
                    fmt.Println("    🔄 This error can be retried")
                }
            } else {
                fmt.Printf("❌ %v\n", err)
            }
        } else {
            fmt.Printf("✅ Success: %s\n", string(data))
        }
    }
    
    // 3. 業務邏輯錯誤測試
    fmt.Println("\n3. 業務邏輯錯誤測試:")
    orders := [][]interface{}{
        {"ORDER-001", 100.50, 123},     // 有效
        {"", 50.00, 456},               // 無效訂單 ID
        {"ORDER-002", -10.00, 789},     // 無效金額
        {"ORDER-003", 15000.00, 101},   // 超過限額
        {"ORDER-004", 200.00, -1},      // 無效客戶 ID
    }
    
    for i, order := range orders {
        orderID := order[0].(string)
        amount := order[1].(float64)
        customerID := order[2].(int)
        
        fmt.Printf("  Order %d (%s): ", i+1, orderID)
        if err := processOrder(orderID, amount, customerID); err != nil {
            if be, ok := err.(BusinessError); ok {
                fmt.Printf("❌ BusinessError - Code: %s\n", be.GetCode())
                if details := be.GetDetails(); len(details) > 0 {
                    fmt.Printf("    Details: %+v\n", details)
                }
            } else {
                fmt.Printf("❌ %v\n", err)
            }
        } else {
            fmt.Println("✅ Order processed successfully")
        }
    }
    
    // 4. 多重錯誤測試
    fmt.Println("\n4. 多重錯誤測試:")
    testUsers := []map[string]interface{}{
        {"name": "Alice", "age": 25, "email": "alice@example.com"},  // 有效
        {"age": 30, "email": "bob@example.com"},                     // 缺少名稱
        {"name": "C", "age": "invalid", "email": "invalid-email"},   // 多個錯誤
        {},  // 空資料
    }
    
    for i, userData := range testUsers {
        fmt.Printf("  User %d: ", i+1)
        if err := validateUserData(userData); err != nil {
            if me, ok := err.(MultiError); ok {
                fmt.Printf("❌ MultiError (%d errors):\n", len(me.Errors))
                for j, subErr := range me.Errors {
                    fmt.Printf("    %d. %v\n", j+1, subErr)
                }
            } else {
                fmt.Printf("❌ %v\n", err)
            }
        } else {
            fmt.Println("✅ Valid user data")
        }
        fmt.Println()
    }
    
    fmt.Println("=== 自訂錯誤型態的優點 ===")
    fmt.Println("✓ 攜帶結構化的錯誤資訊")
    fmt.Println("✓ 支援型態斷言來存取特定欄位")
    fmt.Println("✓ 可以添加自訂方法")
    fmt.Println("✓ 提供更好的錯誤分類和處理")
    fmt.Println("✓ 支援複雜的錯誤情境")
}

## 5. 包裝錯誤

Go 1.13 引入了錯誤包裝功能，允許我們在保留原始錯誤的同時添加上下文資訊。

### 錯誤包裝的使用

In [ ]:
package main

import (
    "errors"
    "fmt"
    "os"
    "strconv"
)

// 模擬不同層級的函式調用

// 底層函式 - 檔案操作
func readConfigFile(filename string) (map[string]string, error) {
    // 模擬檔案不存在的情況
    if filename == "nonexistent.conf" {
        return nil, os.ErrNotExist
    }
    
    // 模擬權限錯誤
    if filename == "protected.conf" {
        return nil, os.ErrPermission
    }
    
    // 模擬格式錯誤
    if filename == "invalid.conf" {
        return nil, fmt.Errorf("invalid config format in line 5")
    }
    
    // 成功情況
    return map[string]string{
        "host": "localhost",
        "port": "8080",
    }, nil
}

// 中間層函式 - 配置解析
func parseConfig(filename string) (map[string]interface{}, error) {
    rawConfig, err := readConfigFile(filename)
    if err != nil {
        return nil, fmt.Errorf("failed to read config file '%s': %w", filename, err)
    }
    
    config := make(map[string]interface{})
    
    for key, value := range rawConfig {
        switch key {
        case "port":
            port, err := strconv.Atoi(value)
            if err != nil {
                return nil, fmt.Errorf("failed to parse port '%s': %w", value, err)
            }
            config[key] = port
        default:
            config[key] = value
        }
    }
    
    return config, nil
}

// 高層函式 - 應用程式初始化
func initializeApp(configFile string) error {
    config, err := parseConfig(configFile)
    if err != nil {
        return fmt.Errorf("application initialization failed: %w", err)
    }
    
    // 驗證必要的配置
    if _, exists := config["host"]; !exists {
        return fmt.Errorf("missing required config: host")
    }
    
    if port, exists := config["port"]; !exists {
        return fmt.Errorf("missing required config: port")
    } else if portInt, ok := port.(int); !ok || portInt <= 0 {
        return fmt.Errorf("invalid port configuration: %v", port)
    }
    
    fmt.Printf("✅ App initialized with config: %+v\n", config)
    return nil
}

// 展示錯誤鏈的函式
func demonstrateErrorChain(err error) {
    fmt.Printf("Error chain analysis:\n")
    fmt.Printf("  Main error: %v\n", err)
    
    // 使用 errors.Unwrap 遍歷錯誤鏈
    level := 1
    for unwrapped := errors.Unwrap(err); unwrapped != nil; unwrapped = errors.Unwrap(unwrapped) {
        fmt.Printf("  Level %d: %v\n", level, unwrapped)
        level++
    }
    
    // 檢查特定錯誤
    fmt.Printf("\nError type checks:\n")
    if errors.Is(err, os.ErrNotExist) {
        fmt.Println("  ✓ Contains os.ErrNotExist")
    }
    if errors.Is(err, os.ErrPermission) {
        fmt.Println("  ✓ Contains os.ErrPermission")
    }
    
    // 檢查數字解析錯誤
    var numErr *strconv.NumError
    if errors.As(err, &numErr) {
        fmt.Printf("  ✓ Contains strconv.NumError: %v\n", numErr)
    }
}

// 自訂包裝錯誤型態
type WrappedError struct {
    Context string
    Err     error
}

func (we WrappedError) Error() string {
    return fmt.Sprintf("%s: %v", we.Context, we.Err)
}

func (we WrappedError) Unwrap() error {
    return we.Err
}

// 使用自訂包裝錯誤的函式
func processData(data string) error {
    if data == "" {
        baseErr := errors.New("empty data")
        return WrappedError{
            Context: "data validation failed",
            Err:     baseErr,
        }
    }
    
    if data == "invalid" {
        baseErr := errors.New("invalid format")
        return WrappedError{
            Context: "data processing failed",
            Err:     baseErr,
        }
    }
    
    return nil
}

// 多層包裝示例
func complexOperation(step int) error {
    switch step {
    case 1:
        err := errors.New("database connection failed")
        return fmt.Errorf("step 1 failed: %w", err)
    case 2:
        err := fmt.Errorf("step 1 failed: %w", errors.New("database connection failed"))
        return fmt.Errorf("step 2 failed: %w", err)
    case 3:
        err := fmt.Errorf("step 2 failed: %w", 
                fmt.Errorf("step 1 failed: %w", errors.New("database connection failed")))
        return fmt.Errorf("step 3 failed: %w", err)
    default:
        return nil
    }
}

func main() {
    fmt.Println("=== 錯誤包裝範例 ===")
    
    // 1. 基本錯誤包裝
    fmt.Println("\n1. 基本錯誤包裝測試:")
    configFiles := []string{
        "app.conf",        // 成功
        "nonexistent.conf", // 檔案不存在
        "protected.conf",   // 權限錯誤
        "invalid.conf",     // 格式錯誤
    }
    
    for _, configFile := range configFiles {
        fmt.Printf("\n--- Testing config file: %s ---\n", configFile)
        
        err := initializeApp(configFile)
        if err != nil {
            fmt.Printf("❌ %v\n\n", err)
            demonstrateErrorChain(err)
        }
        fmt.Println()
    }
    
    // 2. 自訂包裝錯誤
    fmt.Println("\n2. 自訂包裝錯誤測試:")
    testData := []string{"valid", "", "invalid"}
    
    for _, data := range testData {
        fmt.Printf("\n--- Processing data: '%s' ---\n", data)
        err := processData(data)
        if err != nil {
            fmt.Printf("❌ %v\n", err)
            
            // 檢查是否為自訂包裝錯誤
            var wrapped WrappedError
            if errors.As(err, &wrapped) {
                fmt.Printf("  Context: %s\n", wrapped.Context)
                fmt.Printf("  Original error: %v\n", wrapped.Err)
            }
        } else {
            fmt.Println("✅ Data processed successfully")
        }
    }
    
    // 3. 多層包裝錯誤
    fmt.Println("\n3. 多層包裝錯誤測試:")
    for step := 1; step <= 3; step++ {
        fmt.Printf("\n--- Complex operation step %d ---\n", step)
        err := complexOperation(step)
        if err != nil {
            fmt.Printf("❌ %v\n\n", err)
            demonstrateErrorChain(err)
        }
        fmt.Println()
    }
    
    // 4. 錯誤包裝的實際應用
    fmt.Println("\n4. 錯誤包裝實際應用:")
    
    // 模擬一個複雜的錯誤情境
    originalErr := errors.New("connection refused")
    networkErr := fmt.Errorf("network error: %w", originalErr)
    serviceErr := fmt.Errorf("service unavailable: %w", networkErr)
    apiErr := fmt.Errorf("API call failed: %w", serviceErr)
    
    fmt.Printf("Complete error: %v\n\n", apiErr)
    demonstrateErrorChain(apiErr)
    
    fmt.Println("\n=== 錯誤包裝的最佳實踐 ===")
    fmt.Println("✓ 使用 %w 動詞來包裝錯誤")
    fmt.Println("✓ 添加有意義的上下文資訊")
    fmt.Println("✓ 保持錯誤鏈的完整性")
    fmt.Println("✓ 使用 errors.Is 和 errors.As 檢查包裝的錯誤")
    fmt.Println("✓ 避免過度包裝，保持錯誤訊息的清晰")
}

## 6. Is 與 As

Go 1.13 引入了 `errors.Is` 和 `errors.As` 函式來處理包裝的錯誤。這些函式可以檢查錯誤鏈中的特定錯誤。

### errors.Is 和 errors.As 的使用

In [ ]:
package main

import (
    "errors"
    "fmt"
    "net"
    "os"
    "strconv"
    "syscall"
)

// 自訂錯誤型態
type APIError struct {
    Code    int
    Message string
    Service string
}

func (ae APIError) Error() string {
    return fmt.Sprintf("API Error %d: %s (service: %s)", ae.Code, ae.Message, ae.Service)
}

type DatabaseError struct {
    Query     string
    Table     string
    Operation string
}

func (de DatabaseError) Error() string {
    return fmt.Sprintf("Database %s failed on table '%s': %s", de.Operation, de.Table, de.Query)
}

// Sentinel 錯誤
var (
    ErrUserNotFound    = errors.New("user not found")
    ErrInvalidPassword = errors.New("invalid password")
    ErrRateLimited     = errors.New("rate limited")
    ErrMaintenanceMode = errors.New("service in maintenance mode")
)

// 模擬不同的錯誤情況
func authenticateUser(username, password string) error {
    if username == "" {
        return fmt.Errorf("username validation failed: %w", 
                        fmt.Errorf("empty username: %w", ErrUserNotFound))
    }
    
    if username == "nonexistent" {
        dbErr := DatabaseError{
            Query:     "SELECT * FROM users WHERE username = ?",
            Table:     "users",
            Operation: "SELECT",
        }
        return fmt.Errorf("user lookup failed: %w", 
                        fmt.Errorf("database query failed: %w", dbErr))
    }
    
    if password == "wrong" {
        return fmt.Errorf("authentication failed: %w", ErrInvalidPassword)
    }
    
    if username == "ratelimited" {
        apiErr := APIError{
            Code:    429,
            Message: "Too many requests",
            Service: "auth-service",
        }
        return fmt.Errorf("authentication rate limited: %w", 
                        fmt.Errorf("API call failed: %w", apiErr))
    }
    
    return nil
}

func parseNumber(s string) (int, error) {
    if s == "" {
        return 0, fmt.Errorf("parsing failed: %w", errors.New("empty string"))
    }
    
    num, err := strconv.Atoi(s)
    if err != nil {
        return 0, fmt.Errorf("number parsing failed for '%s': %w", s, err)
    }
    
    return num, nil
}

func connectToService(address string) error {
    if address == "timeout.com:80" {
        return fmt.Errorf("connection failed: %w", 
                        &net.OpError{
                            Op:  "dial",
                            Net: "tcp",
                            Addr: &net.TCPAddr{IP: net.IPv4(127, 0, 0, 1), Port: 80},
                            Err:  syscall.ECONNREFUSED,
                        })
    }
    
    if address == "maintenance.com:80" {
        return fmt.Errorf("service connection failed: %w", ErrMaintenanceMode)
    }
    
    return nil
}

// 演示 errors.Is 的使用
func demonstrateErrorsIs(err error) {
    fmt.Printf("\n--- errors.Is Analysis ---\n")
    fmt.Printf("Error: %v\n", err)
    
    // 檢查 Sentinel 錯誤
    sentinelChecks := map[string]error{
        "ErrUserNotFound":    ErrUserNotFound,
        "ErrInvalidPassword": ErrInvalidPassword,
        "ErrRateLimited":     ErrRateLimited,
        "ErrMaintenanceMode": ErrMaintenanceMode,
    }
    
    for name, sentinel := range sentinelChecks {
        if errors.Is(err, sentinel) {
            fmt.Printf("  ✓ Contains %s\n", name)
        }
    }
    
    // 檢查標準函式庫錯誤
    if errors.Is(err, os.ErrNotExist) {
        fmt.Println("  ✓ Contains os.ErrNotExist")
    }
    if errors.Is(err, syscall.ECONNREFUSED) {
        fmt.Println("  ✓ Contains syscall.ECONNREFUSED")
    }
}

// 演示 errors.As 的使用
func demonstrateErrorsAs(err error) {
    fmt.Printf("\n--- errors.As Analysis ---\n")
    fmt.Printf("Error: %v\n", err)
    
    // 檢查 APIError
    var apiErr APIError
    if errors.As(err, &apiErr) {
        fmt.Printf("  ✓ Contains APIError:\n")
        fmt.Printf("    Code: %d\n", apiErr.Code)
        fmt.Printf("    Message: %s\n", apiErr.Message)
        fmt.Printf("    Service: %s\n", apiErr.Service)
    }
    
    // 檢查 DatabaseError
    var dbErr DatabaseError
    if errors.As(err, &dbErr) {
        fmt.Printf("  ✓ Contains DatabaseError:\n")
        fmt.Printf("    Operation: %s\n", dbErr.Operation)
        fmt.Printf("    Table: %s\n", dbErr.Table)
        fmt.Printf("    Query: %s\n", dbErr.Query)
    }
    
    // 檢查 strconv.NumError
    var numErr *strconv.NumError
    if errors.As(err, &numErr) {
        fmt.Printf("  ✓ Contains strconv.NumError:\n")
        fmt.Printf("    Func: %s\n", numErr.Func)
        fmt.Printf("    Num: %s\n", numErr.Num)
        fmt.Printf("    Err: %v\n", numErr.Err)
    }
    
    // 檢查 net.OpError
    var opErr *net.OpError
    if errors.As(err, &opErr) {
        fmt.Printf("  ✓ Contains net.OpError:\n")
        fmt.Printf("    Op: %s\n", opErr.Op)
        fmt.Printf("    Net: %s\n", opErr.Net)
        fmt.Printf("    Addr: %v\n", opErr.Addr)
        fmt.Printf("    Err: %v\n", opErr.Err)
    }
}

// 實際應用：根據錯誤類型決定處理策略
func handleError(err error, operation string) {
    fmt.Printf("\n=== Handling Error for %s ===\n", operation)
    
    // 使用 errors.Is 檢查特定錯誤並決定處理策略
    switch {
    case errors.Is(err, ErrRateLimited):
        fmt.Println("🔄 Strategy: Wait and retry with exponential backoff")
        
    case errors.Is(err, ErrMaintenanceMode):
        fmt.Println("⏳ Strategy: Schedule retry for later")
        
    case errors.Is(err, ErrUserNotFound):
        fmt.Println("❌ Strategy: Show 'User not found' message to user")
        
    case errors.Is(err, ErrInvalidPassword):
        fmt.Println("🔒 Strategy: Show 'Invalid credentials' message")
        
    case errors.Is(err, syscall.ECONNREFUSED):
        fmt.Println("🌐 Strategy: Try alternative endpoint")
        
    default:
        fmt.Println("❓ Strategy: Log error and show generic message")
    }
    
    // 使用 errors.As 獲取詳細資訊
    var apiErr APIError
    if errors.As(err, &apiErr) {
        if apiErr.Code >= 500 {
            fmt.Printf("🚨 Server error detected (code %d) - escalate to monitoring\n", apiErr.Code)
        }
    }
    
    var dbErr DatabaseError
    if errors.As(err, &dbErr) {
        fmt.Printf("💾 Database error in %s operation - check query: %s\n", 
                  dbErr.Operation, dbErr.Query)
    }
}

func main() {
    fmt.Println("=== errors.Is 和 errors.As 範例 ===")
    
    // 1. 認證錯誤測試
    fmt.Println("\n1. 認證錯誤測試:")
    authTests := [][]string{
        {"", "password"},          // 空使用者名稱
        {"nonexistent", "password"}, // 不存在的使用者
        {"alice", "wrong"},        // 錯誤密碼
        {"ratelimited", "password"}, // 速率限制
        {"alice", "correct"},      // 成功
    }
    
    for _, test := range authTests {
        username, password := test[0], test[1]
        fmt.Printf("\n--- Authenticating user: '%s' ---\n", username)
        
        err := authenticateUser(username, password)
        if err != nil {
            demonstrateErrorsIs(err)
            demonstrateErrorsAs(err)
            handleError(err, "Authentication")
        } else {
            fmt.Println("✅ Authentication successful")
        }
    }
    
    // 2. 數字解析錯誤測試
    fmt.Println("\n\n2. 數字解析錯誤測試:")
    parseTests := []string{"", "abc", "123.45", "42"}
    
    for _, test := range parseTests {
        fmt.Printf("\n--- Parsing: '%s' ---\n", test)
        
        _, err := parseNumber(test)
        if err != nil {
            demonstrateErrorsIs(err)
            demonstrateErrorsAs(err)
            handleError(err, "Number Parsing")
        } else {
            fmt.Println("✅ Parsing successful")
        }
    }
    
    // 3. 網路連接錯誤測試
    fmt.Println("\n\n3. 網路連接錯誤測試:")
    connectionTests := []string{
        "example.com:80",
        "timeout.com:80",
        "maintenance.com:80",
    }
    
    for _, test := range connectionTests {
        fmt.Printf("\n--- Connecting to: %s ---\n", test)
        
        err := connectToService(test)
        if err != nil {
            demonstrateErrorsIs(err)
            demonstrateErrorsAs(err)
            handleError(err, "Network Connection")
        } else {
            fmt.Println("✅ Connection successful")
        }
    }
    
    fmt.Println("\n\n=== errors.Is 和 errors.As 的使用原則 ===")
    fmt.Println("✓ errors.Is: 檢查錯誤鏈中是否包含特定的錯誤值")
    fmt.Println("✓ errors.As: 從錯誤鏈中提取特定型態的錯誤")
    fmt.Println("✓ 這些函式會遍歷整個錯誤鏈")
    fmt.Println("✓ 適用於包裝的錯誤結構")
    fmt.Println("✓ 支援更精確的錯誤處理策略")
}

## 7. panic 與 recover

雖然 Go 主要使用錯誤值來處理錯誤，但也提供了 `panic` 和 `recover` 機制來處理程式中的嚴重錯誤。

### panic 和 recover 的使用

In [ ]:
package main

import (
    "fmt"
    "runtime"
    "strings"
)

// 展示基本的 panic 情況
func demonstratePanic(scenario string) {
    defer func() {
        if r := recover(); r != nil {
            fmt.Printf("❌ Recovered from panic in %s: %v\n", scenario, r)
        }
    }()
    
    fmt.Printf("\n--- Testing %s ---\n", scenario)
    
    switch scenario {
    case "nil pointer dereference":
        var ptr *int
        fmt.Println(*ptr)  // 這會造成 panic
        
    case "array out of bounds":
        arr := [3]int{1, 2, 3}
        fmt.Println(arr[5])  // 這會造成 panic
        
    case "type assertion failure":
        var i interface{} = "string"
        num := i.(int)  // 這會造成 panic
        fmt.Println(num)
        
    case "manual panic":
        panic("this is a manual panic")
        
    case "division by zero":
        a := 10
        b := 0
        fmt.Println(a / b)  // 整數除法不會 panic，但會給出錯誤結果
        
    default:
        fmt.Println("✅ No panic in this scenario")
    }
    
    fmt.Printf("✅ %s completed successfully\n", scenario)
}

// 安全的除法函式
func safeDivide(a, b float64) (result float64, err error) {
    defer func() {
        if r := recover(); r != nil {
            err = fmt.Errorf("panic occurred during division: %v", r)
            result = 0
        }
    }()
    
    if b == 0 {
        panic("division by zero is not allowed")
    }
    
    result = a / b
    return result, nil
}

// 安全的陣列存取
func safeArrayAccess(arr []int, index int) (value int, err error) {
    defer func() {
        if r := recover(); r != nil {
            err = fmt.Errorf("array access panic: %v", r)
            value = 0
        }
    }()
    
    if index < 0 || index >= len(arr) {
        panic(fmt.Sprintf("index %d out of bounds for array of length %d", index, len(arr)))
    }
    
    value = arr[index]
    return value, nil
}

// 巢狀 defer 和 recover
func nestedPanicExample() {
    defer func() {
        if r := recover(); r != nil {
            fmt.Printf("🔧 Outer recover caught: %v\n", r)
        }
    }()
    
    fmt.Println("\n--- Nested Panic Example ---")
    
    func() {
        defer func() {
            if r := recover(); r != nil {
                fmt.Printf("🔧 Inner recover caught: %v\n", r)
                // 重新 panic
                panic(fmt.Sprintf("re-panicking: %v", r))
            }
        }()
        
        panic("original panic")
    }()
    
    fmt.Println("This line will not be reached")
}

// 資源清理的 defer 使用
func resourceCleanupExample() {
    fmt.Println("\n--- Resource Cleanup Example ---")
    
    // 模擬資源分配
    resource1 := "Database Connection"
    resource2 := "File Handle"
    resource3 := "Network Socket"
    
    defer func() {
        fmt.Printf("🧹 Cleaning up %s\n", resource3)
    }()
    
    defer func() {
        fmt.Printf("🧹 Cleaning up %s\n", resource2)
    }()
    
    defer func() {
        fmt.Printf("🧹 Cleaning up %s\n", resource1)
        if r := recover(); r != nil {
            fmt.Printf("🚨 Error during cleanup, but recovered: %v\n", r)
        }
    }()
    
    fmt.Println("📦 Resources allocated")
    
    // 模擬工作過程中的 panic
    if true {  // 改成 false 來測試正常流程
        panic("simulated error during work")
    }
    
    fmt.Println("✅ Work completed successfully")
}

// 取得 panic 的堆疊追蹤
func panicWithStackTrace(depth int) {
    defer func() {
        if r := recover(); r != nil {
            fmt.Printf("\n🔍 Panic recovered: %v\n", r)
            
            // 取得堆疊追蹤
            buf := make([]byte, 1024)
            n := runtime.Stack(buf, false)
            fmt.Printf("Stack trace:\n%s\n", string(buf[:n]))
        }
    }()
    
    if depth <= 0 {
        panic("reached maximum depth")
    }
    
    panicWithStackTrace(depth - 1)
}

// 實際應用：HTTP 伺服器的 panic 處理
func httpHandlerSimulation(path string) {
    defer func() {
        if r := recover(); r != nil {
            fmt.Printf("🌐 HTTP Handler recovered from panic for path %s: %v\n", path, r)
            // 在實際應用中，這裡會回傳 500 Internal Server Error
            fmt.Println("📤 Returning 500 Internal Server Error to client")
        }
    }()
    
    fmt.Printf("\n🌐 Handling request for path: %s\n", path)
    
    switch path {
    case "/panic":
        panic("handler panic for testing")
    case "/error":
        var slice []string
        fmt.Println(slice[10])  // index out of range
    case "/nil":
        var m map[string]int
        m["key"] = 1  // nil map panic
    default:
        fmt.Printf("✅ Successfully handled %s\n", path)
    }
}

// 檢查 panic 是否應該被恢復
func smartRecover(operation string) {
    defer func() {
        if r := recover(); r != nil {
            // 將 panic 轉換為字串進行分析
            panicStr := fmt.Sprintf("%v", r)
            
            // 某些 panic 不應該被恢復
            if strings.Contains(panicStr, "runtime error") {
                fmt.Printf("🚨 Critical runtime error in %s, not recovering: %v\n", operation, r)
                // 重新 panic
                panic(r)
            }
            
            fmt.Printf("🔧 Recovered from recoverable panic in %s: %v\n", operation, r)
        }
    }()
    
    fmt.Printf("\n--- Smart Recover for %s ---\n", operation)
    
    switch operation {
    case "business logic":
        panic("business rule violation")  // 可恢復
    case "runtime error":
        var ptr *int
        *ptr = 42  // runtime error，不應恢復
    }
}

func main() {
    fmt.Println("=== panic 和 recover 範例 ===")
    
    // 1. 基本 panic 情況
    fmt.Println("\n1. 基本 panic 情況:")
    scenarios := []string{
        "nil pointer dereference",
        "array out of bounds",
        "type assertion failure",
        "manual panic",
        "normal operation",
    }
    
    for _, scenario := range scenarios {
        demonstratePanic(scenario)
    }
    
    // 2. 安全的函式調用
    fmt.Println("\n\n2. 安全的函式調用:")
    
    // 安全除法測試
    fmt.Println("\n--- Safe Division Tests ---")
    divisionTests := [][]float64{
        {10, 2},   // 正常
        {7, 0},    // 除零
        {-5, 2.5}, // 負數
    }
    
    for _, test := range divisionTests {
        a, b := test[0], test[1]
        result, err := safeDivide(a, b)
        if err != nil {
            fmt.Printf("  %.1f / %.1f: ❌ %v\n", a, b, err)
        } else {
            fmt.Printf("  %.1f / %.1f = %.2f ✅\n", a, b, result)
        }
    }
    
    // 安全陣列存取測試
    fmt.Println("\n--- Safe Array Access Tests ---")
    arr := []int{10, 20, 30, 40, 50}
    indices := []int{0, 2, 5, -1, 10}
    
    for _, index := range indices {
        value, err := safeArrayAccess(arr, index)
        if err != nil {
            fmt.Printf("  arr[%d]: ❌ %v\n", index, err)
        } else {
            fmt.Printf("  arr[%d] = %d ✅\n", index, value)
        }
    }
    
    // 3. 巢狀 panic 和 recover
    fmt.Println("\n3. 巢狀 panic 和 recover:")
    nestedPanicExample()
    
    // 4. 資源清理
    fmt.Println("\n4. 資源清理:")
    resourceCleanupExample()
    
    // 5. 堆疊追蹤
    fmt.Println("\n5. 堆疊追蹤:")
    panicWithStackTrace(3)
    
    // 6. HTTP 處理器模擬
    fmt.Println("\n6. HTTP 處理器模擬:")
    paths := []string{"/home", "/panic", "/error", "/nil"}
    for _, path := range paths {
        httpHandlerSimulation(path)
    }
    
    // 7. 智慧型恢復
    fmt.Println("\n7. 智慧型恢復:")
    
    // 測試可恢復的 panic
    func() {
        defer func() {
            if r := recover(); r != nil {
                fmt.Printf("🔧 Main recovered: %v\n", r)
            }
        }()
        smartRecover("business logic")
    }()
    
    // 測試不可恢復的 panic（這會導致程式終止）
    // smartRecover("runtime error")  // 取消註解來測試
    
    fmt.Println("\n=== panic 和 recover 的使用原則 ===")
    fmt.Println("✓ panic 應該用於真正無法恢復的錯誤")
    fmt.Println("✓ recover 只能在 defer 函式中使用")
    fmt.Println("✓ 優先使用錯誤值而不是 panic")
    fmt.Println("✓ defer 確保資源清理")
    fmt.Println("✓ 謹慎選擇是否要恢復 panic")
    fmt.Println("✓ 記錄 panic 資訊以供除錯")
}

## 總結

本章詳細介紹了 Go 語言的錯誤處理機制，從基礎概念到進階技巧：

### 重點回顧

1. **基本錯誤處理**：Go 將錯誤視為值，使用 `error` 介面
2. **錯誤建立**：使用 `errors.New()` 和 `fmt.Errorf()` 建立錯誤
3. **Sentinel 錯誤**：預定義的錯誤值，便於特定錯誤檢查
4. **自訂錯誤型態**：攜帶結構化資訊的錯誤
5. **錯誤包裝**：使用 `%w` 動詞保留錯誤鏈
6. **errors.Is 和 errors.As**：檢查和提取包裝錯誤中的特定錯誤
7. **panic 和 recover**：處理程式中的嚴重錯誤

### 最佳實踐

- **總是檢查錯誤**：`if err != nil`
- **提供有意義的錯誤訊息**：包含足夠的上下文
- **使用錯誤包裝**：保留錯誤鏈的完整性
- **適當使用 Sentinel 錯誤**：便於錯誤分類處理
- **優先使用錯誤值**：而不是 panic
- **謹慎使用 recover**：只在必要時恢復 panic

### 注意事項

- 錯誤處理會讓程式碼變長，但提高了可靠性
- 避免忽略錯誤，即使是在範例程式碼中
- panic 應該用於真正無法恢復的情況
- 錯誤訊息應該對除錯有幫助
- 使用 `errors.Is` 和 `errors.As` 而不是直接比較錯誤